# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in csv file
df_weatherpy = pd.read_csv('../WeatherPy/Weather_Data.csv').drop('Unnamed: 0', axis=1)
df_weatherpy.head()

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Dikson,73.51,80.55,30.72,94,100,20.83,RU,1602798371
1,Hithadhoo,-0.60,73.08,82.31,75,97,20.09,MV,1602798372
2,Moerai,-22.43,-151.33,72.18,84,88,18.54,PF,1602798372
3,George Town,5.41,100.34,77.00,88,40,8.05,MY,1602798243
4,Punta Arenas,-53.15,-70.92,51.80,53,75,34.45,CL,1602798373


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create location df and humidity series for gmaps
locations = df_weatherpy[['Lat', 'Lng']].astype(float)
humidity = df_weatherpy['Humidity (%)'].astype(float)

In [4]:
# Create humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create a new df to narrow down resules to ideal conditions for vacation.
df_vacay = df_weatherpy.loc[(df_weatherpy['Max Temp (F)'] > 70) & (df_weatherpy['Max Temp (F)'] < 80) & \
                            (df_weatherpy['Wind Speed (mph)'] < 10) & (df_weatherpy['Cloudiness (%)'] == 0) & \
                            (df_weatherpy['Humidity (%)'] < 70)]
df_vacay

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
28,Port Hedland,-20.32,118.57,73.40,56,0,9.17,AU,1602798107
149,Alice Springs,-23.70,133.88,75.00,52,0,4.70,AU,1602798438
204,Zaria,11.11,7.72,78.24,50,0,2.24,NG,1602798461
225,Buraidah,26.33,43.98,75.20,27,0,2.24,SA,1602798356
342,Sakakah,29.97,40.21,73.40,27,0,5.82,SA,1602798507
376,Karratha,-20.74,116.85,72.93,38,0,9.17,AU,1602798518
412,Kutum,14.20,24.67,78.51,18,0,9.78,SD,1602798530


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create new df from narrowed results
df_hotel = pd.DataFrame(df_vacay, columns = ['City', 'Country', 'Lat', 'Lng'])
df_hotel['Hotel Name'] = ""
df_hotel

,City,Country,Lat,Lng,Hotel Name
28,Port Hedland,AU,-20.32,118.57,
149,Alice Springs,AU,-23.70,133.88,
204,Zaria,NG,11.11,7.72,
225,Buraidah,SA,26.33,43.98,
342,Sakakah,SA,29.97,40.21,
376,Karratha,AU,-20.74,116.85,
412,Kutum,SD,14.20,24.67,


In [7]:
# To find the nearest hotel - 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": 'hotel',  
    "type": 'hotel',
    "radius": 5000,
    "key": g_key,}

# Use iterrows to iterate through the pandas df
for index, row in df_hotel.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    # Add keyword to params dict
    params['location'] = f"{lat},{lng}"
    
    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results =  response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        df_hotel.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing result... skipping.")
    print("------------")

Retrieving Results for Index 28: Port Hedland.
Closest hotel in Port Hedland is The Esplanade Hotel.
------------
Retrieving Results for Index 149: Alice Springs.
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
------------
Retrieving Results for Index 204: Zaria.
Closest hotel in Zaria is Sleeprite Hotels.
------------
Retrieving Results for Index 225: Buraidah.
Closest hotel in Buraidah is فندق براند القصيم.
------------
Retrieving Results for Index 342: Sakakah.
Closest hotel in Sakakah is Raoum Inn Hotel.
------------
Retrieving Results for Index 376: Karratha.
Closest hotel in Karratha is Karratha International Hotel.
------------
Retrieving Results for Index 412: Kutum.
Missing result... skipping.
------------


In [8]:
df_hotel

,City,Country,Lat,Lng,Hotel Name
28,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
149,Alice Springs,AU,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
204,Zaria,NG,11.11,7.72,Sleeprite Hotels
225,Buraidah,SA,26.33,43.98,فندق براند القصيم
342,Sakakah,SA,29.97,40.21,Raoum Inn Hotel
376,Karratha,AU,-20.74,116.85,Karratha International Hotel
412,Kutum,SD,14.20,24.67,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations = df_hotel[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))